In [14]:
# 0. Instalação da dependência (Rode uma vez no Colab)
!pip install xlsxwriter

import pandas as pd
from datetime import datetime, timedelta
from abc import ABC, abstractmethod

# 1. ARQUITETURA POO (ABSTRAÇÃO E HERANÇA)
class Imovel(ABC):
    def __init__(self, tipo):
        self.tipo = tipo
        self.valor_contrato = 2000.00
        self.iptu, self.seguro = 85.00, 35.00

    @abstractmethod
    def calcular_aluguel(self):
        pass

class Apartamento(Imovel):
    def __init__(self, q, g, c):
        super().__init__("Apartamento")
        self.q = q
        self.g = g
        self.c = c

    def calcular_aluguel(self):
        v = 700.00
        if self.q >= 2:
            v += 200.00
        if self.g:
            v += 300.00
        if not self.c:
            v *= 0.95
        return v, 320.00

class Casa(Imovel):
    def __init__(self, q, g):
        super().__init__("Casa")
        self.q = q
        self.g = g

    def calcular_aluguel(self):
        v = 900.00
        if self.q >= 2:
            v += 250.00
        if self.g:
            v += 300.00
        return v, 0.0

class Estudio(Imovel):
    def __init__(self, v_garagem):
        super().__init__("Estúdio")
        self.v_garagem = v_garagem

    def calcular_aluguel(self):
        v = 1200.00
        if self.v_garagem > 0:
            v += 250.00 + (max(0, self.v_garagem - 2) * 60.00)
        return v, 0.0

# 2. GERADOR DE ORÇAMENTO EM EXCEL (ARIAL 12)
def gerar_planilha_premium(imovel, parcelas_taxa):
    mensalidade, condo = imovel.calcular_aluguel()
    p_adesao = imovel.valor_contrato / parcelas_taxa

    data_ini = datetime.now() + timedelta(days=30)
    dados = []

    for i in range(1, 13):
        venc = (data_ini + timedelta(days=30*(i-1))).strftime("%d/%m/%Y")
        taxa = p_adesao if i <= parcelas_taxa else 0.0
        desc = f"Aluguel" + (f" + Taxa Contrato ({i}/{parcelas_taxa})" if taxa > 0 else "")
        total = mensalidade + taxa + imovel.iptu + imovel.seguro + condo
        dados.append([venc, desc, mensalidade, taxa, imovel.iptu, imovel.seguro, condo, total])

    df = pd.DataFrame(dados, columns=["Vencimento", "Descrição", "Aluguel (R$)", "Taxa Contrato (R$)", "IPTU", "Seguro", "Condo", "Total"])

    nome_arquivo = 'orcamento_final_Leidiane.xlsx'
    writer = pd.ExcelWriter(nome_arquivo, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Orçamento', index=False)
    wb, ws = writer.book, writer.sheets['Orçamento']

    # Estilos Arial 12
    head = wb.add_format({'bold':True, 'font_name':'Arial', 'font_size':12, 'border':1, 'bg_color':'#D9EAD3', 'align':'center'})
    body = wb.add_format({'font_name':'Arial', 'font_size':12, 'border':1, 'align':'center'})
    money = wb.add_format({'font_name':'Arial', 'font_size':12, 'border':1, 'num_format':'R$ #,##0.00'})

    for col, val in enumerate(df.columns):
        ws.write(0, col, val, head)

    ws.set_column('A:A', 15, body)
    ws.set_column('B:B', 45, body)
    ws.set_column('C:H', 20, money)

    # Rodapé com Somas
    ws.write(13, 1, "TOTAL ACUMULADO (12 MESES)", head)
    for c in range(2, 8):
        ws.write_formula(13, c, f'=SUM({chr(65+c)}2:{chr(65+c)}13)', money)

    writer.close()
    print(f"\n✅ Sucesso! Planilha '{nome_arquivo}' gerada na pasta lateral.")

# 3. INTERFACE INTERATIVA
def iniciar_sistema():
    print("--- SISTEMA DE ORÇAMENTOS R.M ---")
    print("1. Apartamento | 2. Casa | 3. Estúdio")

    try:
        escolha = input("Escolha o tipo de imóvel (1, 2 ou 3): ")

        if escolha == '1':
            q = int(input("Número de quartos: "))
            g = input("Vaga de garagem? (s/n): ").lower() == 's'
            c = input("Possui crianças? (s/n): ").lower() == 's'
            imovel = Apartamento(q, g, c)
        elif escolha == '2':
            q = int(input("Número de quartos: "))
            g = input("Vaga de garagem? (s/n): ").lower() == 's'
            imovel = Casa(q, g)
        elif escolha == '3':
            v = int(input("Total de vagas de garagem: "))
            imovel = Estudio(v)
        else:
            print("Opção inválida!"); return

        parc = int(input("Parcelar taxa de contrato (1 a 5x): "))
        if not (1 <= parc <= 5):
            print("Erro: O parcelamento deve ser entre 1 e 5 vezes."); return

        gerar_planilha_premium(imovel, parc)

    except ValueError:
        print("Erro: Por favor, insira apenas números válidos.")

if __name__ == "__main__":
    iniciar_sistema()

--- SISTEMA DE ORÇAMENTOS R.M ---
1. Apartamento | 2. Casa | 3. Estúdio
Escolha o tipo de imóvel (1, 2 ou 3): 1
Número de quartos: 2
Vaga de garagem? (s/n): s
Possui crianças? (s/n): s
Parcelar taxa de contrato (1 a 5x): 5

✅ Sucesso! Planilha 'orcamento_final_Leidiane.xlsx' gerada na pasta lateral.
